In [81]:
import pandas as pd
import numpy as np
import matplotlib as mpl
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline
#%matplotlib notebook

In [82]:
df = pd.read_pickle("properati.pkl")

* Descartamos que las otras columnas de precio nos sirvan para imputar superficies donde falten, y que existan datos de precio cuando price_aprox_usd sea nula

In [83]:
# Hay datos de precio x m2 en filas que no tengan datos de superficie?
df["price_usd_per_m2"].loc[(df["surface_total_in_m2"].isnull() & df["surface_covered_in_m2"].isnull())].count()

0

In [84]:
df["price_per_m2"].loc[(df["surface_total_in_m2"].isnull() & df["surface_covered_in_m2"].isnull())].count()

0

In [85]:
# Hay datos de precio en moneda original para los campos en null en "price_aprox_usd"?
print(df["price"].loc[df["price_aprox_usd"].isnull()].count())
print(df["price_aprox_local_currency"].loc[df["price_aprox_usd"].isnull()].count())

0
0


* Detectamos errores posibles en la carga de la moneda

In [86]:
pricesUSD = df[(
    ~np.isnan(df['price']) & #aca estoy sacando los NaN
    [False if pd.isnull(x) else x=='USD' for x in df['currency']] #aca estoy filtrando los que tengan currency en USD
)].sort_values(by='price_usd_per_m2', ascending=False).head(10);


display('Cantidad de valores en ARS: %d' %len(pricesUSD))
display(pricesUSD['place_name'].unique())

'Cantidad de valores en ARS: 10'

array(['Boedo', 'Ituzaingó', 'Rosario', 'Villa Ballester'], dtype=object)

In [87]:
# reparamos los precios erróneos encontrados
pricesFixed = pricesUSD.copy()
valorDolar = np.round(pricesFixed['price_aprox_local_currency'] / pricesFixed['price_aprox_usd'], 2)

pricesFixed['price_aprox_local_currency'] = pricesFixed['price_aprox_usd']
pricesFixed['price_aprox_usd'] = np.round(pricesFixed['price_aprox_usd'] / valorDolar)
pricesFixed['price'] = pricesFixed['price_aprox_usd']

pricesFixed['price_usd_per_m2'] = pricesFixed['price'] / pricesFixed['surface_covered_in_m2']

pricesFixed.currency = 'ARS'

df.iloc[pricesFixed.index] = pricesFixed

* Imputamos "surface_covered_in_m2" a "surface_total_in_m2" donde la segunda es nula pero hay datos en la primera

In [88]:
# la media de superficie antes de imputar
df["surface_total_in_m2"].mean()

233.79532799296635

In [89]:
# imputamos
df["surface_total_in_m2"] = \
np.where(df["surface_total_in_m2"].isnull(),df["surface_covered_in_m2"],df["surface_total_in_m2"])

In [90]:
# la media de superficie despues de imputar
df["surface_total_in_m2"].mean()

211.39387786974856

In [91]:
# chequeamos que la imputación haya sido completa
df["surface_covered_in_m2"].loc[df["surface_total_in_m2"].isnull()].count()

0

* Parseamos el campo "description" para rescatar datos

In [92]:
# en busca de cantidad de dormitorios
df["dorm_en_desc"] = df['description'].str.extract('(\d+?) (dormitorios|dormitorio)')[0]
# contamos cuantos datos ganamos
print(df["dorm_en_desc"].loc[df["rooms"].isnull()].count())
# imputamos sumando 1 para transformar dormitorios en ambientes
df["rooms"] = np.where(df["rooms"].isnull(),pd.to_numeric(df["dorm_en_desc"]) + 1,df["rooms"])
# dropeamos la columna intermedia
df.drop(columns=["dorm_en_desc"], inplace=True)

17714


In [93]:
# en busca de cantidad de ambientes
df["amb_en_desc"] = df['description'].str.extract('(\d+?) (ambientes|ambiente)')[0]
# contamos cuantos datos ganamos
print(df["amb_en_desc"].loc[df["rooms"].isnull()].count())
# imputamos suma
df["rooms"] = np.where(df["rooms"].isnull(),pd.to_numeric(df["amb_en_desc"]),df["rooms"])
# dropeamos la columna intermedia
df.drop(columns=["amb_en_desc"], inplace=True)

8080


* Parseamos el campo "title" para rescatar datos

In [94]:
# en busca de cantidad de dormitorios
df["dorm_en_tit"] = df['title'].str.extract('(\d+?) (dormitorios|dormitorio)')[0]
# y contamos cuantos datos ganamos
print(df["dorm_en_tit"].loc[df["rooms"].isnull()].count())
# imputamos sumando 1 para transformar dormitorios en ambientes
df["rooms"] = np.where(df["rooms"].isnull(),pd.to_numeric(df["dorm_en_tit"]) + 1,df["rooms"])
# dropeamos la columna intermedia
df.drop(columns=["dorm_en_tit"], inplace=True)

557


In [95]:
# en busca de cantidad de ambientes
df["amb_en_tit"] = df['title'].str.extract('(\d+?) (ambientes|ambiente)')[0]
# contamos cuantos datos ganamos
print(df["amb_en_tit"].loc[df["rooms"].isnull()].count())
# imputamos
df["rooms"] = np.where(df["rooms"].isnull(),pd.to_numeric(df["amb_en_tit"]),df["rooms"])
# dropeamos la columna intermedia
df.drop(columns=["amb_en_tit"], inplace=True)

917


* Eliminamos outliers en "price_aprox_usd" (por zona y tipo) rellenando con nan por encima de los 3 desvios

In [96]:
# transformamos en zscore por state_name, place_name y property_type la variable "price_aprox_usd"
df["price_aprox_usd_zscore"] = (df.groupby(['state_name','place_name','property_type'])[["price_aprox_usd"]]
.apply(lambda x: (x - x.mean()) / x.std()))
# y contamos outliers por sobre los 3 desvíos
df["price_aprox_usd_zscore"][df["price_aprox_usd_zscore"] > 3].count()

1825

In [97]:
# calculamos la media antes de eliminar outliers
df["price_aprox_usd"].mean()

239325.43501795456

In [98]:
#eliminamos outliers
df["price_aprox_usd"] = np.where(df["price_aprox_usd_zscore"]>3,np.nan,df["price_aprox_usd"])

In [99]:
# calculamos la media despues de eliminar outliers
df["price_aprox_usd"].mean()

218745.10077375357

#### --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [102]:
# eliminar los ceros en la variable surface_total_in_m2
df["surface_total_in_m2"] = np.where(df["surface_total_in_m2"] == 0, np.nan,df["surface_total_in_m2"])

In [103]:
# eliminar los ceros en la variable surface_total_in_m2
df["price_aprox_usd"] = np.where(df["price_aprox_usd"] == 0, np.nan,df["price_aprox_usd"])

In [104]:
# recalcular el precio por m2 despues de repoblar las superficies
df["price_usd_m2_nue"] = np.divide(df["price_aprox_usd"],df["surface_total_in_m2"])

In [105]:
df["price_usd_m2_nue"].mean()

2605.6516295838446

In [75]:
# Analizar por zona y tipo de propiedad el valor del m2
df.groupby(['state_name','place_name','property_type'])[["price_usd_m2_nue"]].median().sort_values(by="price_usd_m2_nue",ascending=False)

,,,price_usd_m2_nue
state_name,place_name,property_type,
Bs.As. G.B.A. Zona Oeste,Marcos Paz,store,225140.209175
Bs.As. G.B.A. Zona Sur,GreenVille Polo & Resort,apartment,102250.000000
Bs.As. G.B.A. Zona Norte,"Barrio Cerrado ""La Escondida""",house,38450.000000
Córdoba,Las Calles,house,28000.000000
Bs.As. G.B.A. Zona Oeste,Trujui,store,25000.000000
Entre Ríos,Alejandro Roca,house,24038.461538
Capital Federal,Velez Sarsfield,store,21750.000000
Bs.As. G.B.A. Zona Norte,El Canton Barrio Norte,house,20000.000000
Bs.As. G.B.A. Zona Sur,San Francisco Solano,store,19000.000000


In [76]:
#df["price_aprox_usd"] = np.where(df["price_aprox_usd"] == 0, np.nan,df["price_aprox_usd"])
df.isnull().sum()

Unnamed: 0                         0
operation                          0
property_type                      0
place_name                        23
place_with_parent_names            0
country_name                       0
state_name                         0
geonames_id                    18717
lat-lon                        51550
lat                            51550
lon                            51550
price                          20410
currency                       20411
price_aprox_local_currency     20410
price_aprox_usd                22236
surface_total_in_m2            12752
surface_covered_in_m2          19907
price_usd_per_m2               52603
price_per_m2                   33562
floor                         113321
rooms                          46562
expenses                      106958
properati_url                      0
description                        2
title                              0
image_thumbnail                 3112
price_aprox_usd_zscore         20959
p

In [77]:
df.loc[(df["price_aprox_usd"].notna() & df["surface_total_in_m2"].notna())].count()

Unnamed: 0                    91231
operation                     91231
property_type                 91231
place_name                    91208
place_with_parent_names       91231
country_name                  91231
state_name                    91231
geonames_id                   75665
lat-lon                       53005
lat                           53005
lon                           53005
price                         91231
currency                      91231
price_aprox_local_currency    91231
price_aprox_usd               91231
surface_total_in_m2           91231
surface_covered_in_m2         86041
price_usd_per_m2              67411
price_per_m2                  86039
floor                          6531
rooms                         57828
expenses                      12618
properati_url                 91231
description                   91230
title                         91231
image_thumbnail               89413
price_aprox_usd_zscore        90761
price_usd_m2_nue            

In [80]:
df.groupby(["state_name",'property_type'])[["expenses"]].mean()

expenses
state_name                   property_type               
Bs.As. G.B.A. Zona Norte     PH                613.976744
                             apartment        3021.644444
                             house            5869.713542
                             store            1564.108108
Bs.As. G.B.A. Zona Oeste     PH                266.062500
                             apartment        2818.095041
                             house            3641.730000
                             store            1190.823529
Bs.As. G.B.A. Zona Sur       PH                462.488372
                             apartment        1503.664348
                             house            4589.916974
                             store            1203.214286
Buenos Aires Costa Atlántica PH                718.571429
                             apartment        2042.121065
                             house            3284.692308
                             store            2201.571429
Buenos Aires Interior        PH                620.000000
                             apartment         891.923077
                             house            4508.266667
                             store                    NaN
Capital Federal              PH                962.730769
                             apartment        6634.878631
                             house            2675.916667
                             store            2896.549505
Catamarca                    apartment                NaN
                             house                    NaN
Chaco                        apartment        2100.000000
                             house                    NaN
                             store                    NaN
Chubut                       PH                       NaN
...                                                   ...
Río Negro                    apartment        1096.800000
                             house            2000.500000
                             store            5500.500000
Salta                        PH                       NaN
                             apartment        4100.000000
                             house            2500.000000
                             store                    NaN
San Juan                     apartment                NaN
                             house                    NaN
                             store                    NaN
San Luis                     PH                       NaN
                             apartment         783.333333
                             house            3801.000000
                             store                    NaN
Santa Cruz                   apartment                NaN
                             house                    NaN
                             store                    NaN
Santa Fe                     PH                520.000000
                             apartment        1429.937282
                             house          107839.428571
                             store             812.222222
Santiago Del Estero          apartment                NaN
                             house               1.000000
Tierra Del Fuego             apartment                NaN
                             house             300.000000
                             store                    NaN
Tucumán                      PH               1500.000000
                             apartment        2023.413793
                             house            2679.629630
                             store            4516.250000

[98 rows x 1 columns]

In [79]:
df.loc[(df["price_usd_m2_nue"].notna() & df["property_type"].notna() & df["rooms"].notna())].count()

Unnamed: 0                    57828
operation                     57828
property_type                 57828
place_name                    57817
place_with_parent_names       57828
country_name                  57828
state_name                    57828
geonames_id                   48460
lat-lon                       40465
lat                           40465
lon                           40465
price                         57828
currency                      57828
price_aprox_local_currency    57828
price_aprox_usd               57828
surface_total_in_m2           57828
surface_covered_in_m2         55062
price_usd_per_m2              41490
price_per_m2                  55060
floor                          5361
rooms                         57828
expenses                       6372
properati_url                 57828
description                   57828
title                         57828
image_thumbnail               56993
price_aprox_usd_zscore        57553
price_usd_m2_nue            